# Portfolio Optimization

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim


# Import all remaining packages
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
from scipy.io.wavfile import write

# Batches selection (not important)

In [3]:
def get_batch(dataset, batch_size, seq_length, deltat_in):
    n = dataset.shape[0]
    time_steps = dataset.shape[1]
    idx = np.random.choice(time_steps - seq_length, batch_size)

    '''TODO: construct a list of input sequences for the training batch'''
    input_batch = [dataset[i:i + seq_length] for i in idx]

    '''TODO: construct a list of output sequences for the training batch'''
    output_batch = [dataset[i + deltat_in:i + seq_length + deltat_in] for i in idx]

    # Convert the input and output batches to tensors
    x_batch = torch.tensor(input_batch, dtype=torch.long)
    y_batch = torch.tensor(output_batch, dtype=torch.long)

    return x_batch, y_batch


# Dense Layer

In [ ]:
class DenseLayer(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(DenseLayer, self).__init__()
        # Define and initialize parameters: a weight matrix W and bias b
        # Note that the parameter initialize is random!
        self.W = torch.nn.Parameter(torch.randn(num_inputs, num_outputs))
        self.bias = torch.nn.Parameter(torch.randn(num_outputs))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Implement the forward pass of the dense layer
        return x @ self.W + self.bias

# Lag-transformation module

In [ ]:

class LagTransform(nn.Module):
    def __init__(self):
        super().__init__()

    @staticmethod
    def hyperbolic_weighting(theta, t):
        alpha= theta[0] * t**(-theta[1])
        return alpha

    @staticmethod
    def saturating_exponential_threshold(theta, t):
        beta= theta[2] - theta[3]*torch.exp(-theta[4]*t)
        return beta


    def forward(self, theta: torch.Tensor, t: torch.Tensor, input : torch.Tensor):
        alpha = self.hyperbolic_weighting(theta, t)
        beta  = self.saturating_exponential_threshold(theta, t)
        return alpha/beta * torch.tanh(beta * input)


In [ ]:
model=LagTransform()
y= model(torch.tensor([0.5,0.5,1.,0.5,0.5]), torch.tensor(10.0), torch.tensor([2.0,1.0]))


tensor([0.1529, 0.1206])


# Correlation Cleaning Module

In [ ]:
class CorrelationCleaning:
    def __init__(self):
        super().__init__()
    
    def biGRU(self, input_size, hidden_size, num_layers, dropout):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.bigru = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, bidirectional=True, batch_first=True)
        return self.bigru